# Model CNN Sesuai LeNet-5
Proses pembuatan model CNN akan memanfaatkan subclass dari `nn.Module`. Fungsi dari `Module` ini adalah sebagai wadah berbagai parameter yang akan dilatih. Parameter pada model dapat diakses lewat method `.parameters()`. Selain itu kita hanya perlu implementasi method `forward` dari subclass yang dibuat dan perhitungan gradien `autograd` dapat berjalan secara otomatis seperti pada contoh notebook sebelumnya

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

![](img/03cnn.png)
Langkah-langkah pembuatan CNN  
1. Definisi model beserta parameter sesuai gambar di atas
2. Hitung loss/cost nya dan lakukan backpropagation
4. Update parameter

Untuk menghitung ukuran kernel/filter memakai formula: $n_{in} - [stride * (n_{out} - 1) - (2 * padding)]$ Default pada PyTorch memakai stride 1 dan padding 0

## Definisi Model dan Parameter
LeNet yang dibuat kali ini tidak memakai softmax seperti yang seharusnya. Definisikan layer-layer apa saja yang digunakan pada `__init()__` sehingga pada method `.forward()` hanya perlu menuliskan `Function` apa yang digunakan. Perlu diperhatikan untuk mendefinisikan layer selalu gunakan `torch.nn`. Sedangkan `torch.nn.functional` hanya digunakan ketika membuat layer custom atau fungsi tanpa parameter. Detail contohnya bisa dilihat pada [post forum PyTorch ini](https://discuss.pytorch.org/t/how-to-choose-between-torch-nn-functional-and-torch-nn-module/2800)

In [2]:
class LeNet(nn.Module):
    
    def __init__(self):
        super().__init__() #hanya berfungsi pada python3
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5)
        self.fc1 = nn.Linear(in_features = 16 * 5 * 5, out_features = 120) #dengan bias, karena defaultnya True
        self.fc2 = nn.Linear(in_features = 120, out_features = 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(input = x, kernel_size = (2, 2)) #size dapat memakai rumus di atas dengan memisalkan stride 2
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2) #jika persegi tidak wajib memakai tuple
        x = x.view(x.size(0), -1) #cara ini agak berbeda dengan dokumentasi PyTorch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Arsitektur CNN dapat diketahui dengan menginisiasi kelas dan `print` instance tersebut. Seperti disebutkan di atas, parameter bisa diakses melalui method `.parameters()`

In [18]:
cnn = LeNet()
print(cnn)

print('\nDaftar parameter yang ada pada model: ')
for param in cnn.parameters():
    print(type(param), param.size())

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Daftar parameter yang ada pada model: 
<class 'torch.nn.parameter.Parameter'> torch.Size([6, 1, 5, 5])
<class 'torch.nn.parameter.Parameter'> torch.Size([6])
<class 'torch.nn.parameter.Parameter'> torch.Size([16, 6, 5, 5])
<class 'torch.nn.parameter.Parameter'> torch.Size([16])
<class 'torch.nn.parameter.Parameter'> torch.Size([120, 400])
<class 'torch.nn.parameter.Parameter'> torch.Size([120])
<class 'torch.nn.parameter.Parameter'> torch.Size([84, 120])
<class 'torch.nn.parameter.Parameter'> torch.Size([84])
<class 'torch.nn.parameter.Parameter'> torch.Size([10, 84])
<class 'torch.nn.parameter.Parameter'> torch.Size([10])


Seperti terlihat pada hasil di atas, setiap parameter adalah instance dari kelas `Parameter`, sejenis `Tensor` yang otomatis menjadi atribut pada saat mendefinisikan suatu layer

## Hitung Cost dan Melakukan Backpropagation
Dimulai dengan memproses input tanpa memakai cost function

In [22]:
input_img = torch.randn(1, 1, 32, 32)
output = cnn(input_img)
print(output)

cnn.zero_grad() #menghilangkan gradien yang masih tersimpan seperti yang dijelaskan pada notebook sebelumnya
print('\nSebelum dilakukan operasi propagation\n', cnn.fc3.bias.grad)

output.backward(torch.randn(1, 10)) 
print('\nSetelah dilakukan operasi propagation\n', cnn.fc3.bias.grad)

tensor([[-0.0148, -0.0450, -0.0557, -0.1197,  0.0675,  0.0357,  0.0961, -0.0396,
         -0.0412, -0.0434]], grad_fn=<ThAddmmBackward>)

Sebelum dilakukan operasi propagation
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Setelah dilakukan operasi propagation
 tensor([-1.2412, -1.8414, -0.9265,  0.0479, -1.0891, -0.5778, -0.4866, -0.3193,
        -0.1597, -0.9485])


Perlu diperhatikan bahwa PyTorch hanya mendukung proses secara mini-batch sehingga fungsi dari dimensi yang pertama adalah menyatakan mini-batch 1. Selanjutnya dilakukan backpropagation memakai cost function

In [24]:
output = cnn(input_img)
target = torch.randn(10)
target = target.view(1, -1) #ingat, mini-batch
criterion = nn.MSELoss()

cost = criterion(output, target)
print(cost)

cnn.zero_grad() #menghilangkan gradien yang masih tersimpan seperti yang dijelaskan pada notebook sebelumnya
print('\nSebelum dilakukan operasi propagation\n', cnn.fc3.bias.grad)

cost.backward()
print('\nSetelah dilakukan operasi propagation\n', cnn.fc3.bias.grad)

tensor(1.0095, grad_fn=<MseLossBackward>)

Sebelum dilakukan operasi propagation
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Setelah dilakukan operasi propagation
 tensor([-0.2953, -0.0516,  0.0052, -0.2398, -0.0845, -0.2311, -0.0491, -0.3279,
        -0.2578, -0.1393])


## Update Parameter
Setelah gradien tersimpan pada `.grad` maka selanjutnya adalah melakukan update semua parameter memakai nilai gradien tersebut. Berikut cara melakukan optimisasi memakai SGD

In [ ]:
learning_rate = 0.01
for param in cnn.parameters():
    param.data.sub_(f.grad.data * learning_rate)

Selain bisa dilakukan secara manual seperti di atas, PyTorch juga menyediakan package optimisasi pada `torch.optim`. Contoh pemakaiannya ada di bawah

In [29]:
optimizer = optim.SGD(cnn.parameters(), lr = 1e-3) #membuat optimizer

print('\nSebelum dilakukan update\n', cnn.fc3.bias.data)

#gunakan saat iterasi training
optimizer.zero_grad() #jangan lupa
output = cnn(input_img)
loss = criterion(output, target)
loss.backward()
optimizer.step() #update parameter

print('\nSetelah dilakukan update\n', cnn.fc3.bias.data)


Sebelum dilakukan update
 tensor([-0.0245, -0.0543, -0.0661, -0.0850,  0.0445,  0.0335,  0.0721, -0.0089,
        -0.0781, -0.0942])

Setelah dilakukan update
 tensor([-0.0242, -0.0543, -0.0661, -0.0848,  0.0445,  0.0338,  0.0722, -0.0086,
        -0.0779, -0.0940])
